In [1]:
import pandas as pd
df = pd.read_csv('vaccination_all_tweets.csv')
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [2]:
df = df.drop('user_friends', axis = 1)
df = df.drop('id', axis = 1)
df = df.drop('user_description', axis = 1)
df = df.drop('user_created', axis = 1)
df = df.drop('user_followers', axis = 1)
df = df.drop('user_favourites', axis = 1)
df = df.drop('user_verified', axis = 1)
df = df.drop('source', axis = 1)
df = df.drop('retweets', axis = 1)
df = df.drop('favorites', axis = 1)
df = df.drop('is_retweet', axis = 1)
df = df.drop('user_name', axis = 1)
df = df.drop('user_location', axis = 1)
df = df.drop('date', axis = 1)
df = df.dropna(subset=['text'])
df = df.dropna(subset=['hashtags'])

In [3]:
# Remove new line characters

df["text"] = df['text'].str.replace('[\n]','')
df["text"] = df['text'].str.replace('[\t]','')
# Remove new punctuations
df["text"] = df['text'].str.replace('[^\w\s]','')
df["text"] = df['text'].str.replace('[@]','')
df["text"] = df['text'].str.replace('[#]','')

# Remove distracting single quotes
df["text"] = df['text'].str.replace('[\']','')

In [4]:
df.sample(5)

,text,hashtags
33946,ImranKhan received his first dose of the Chine...,"['ImranKhan', 'Sinopharm', 'Pakistan']"
34329,Since the Pfizer amp Moderna vaccines are 95 e...,"['Pfizer', 'Moderna', 'JohnsonAndJohnsonVaccine']"
21607,POTUS Support in Ocugen joe We need the best v...,"['ocgn', 'Covaxin']"
8301,First vaccine done PfizerBiontech for me Tha...,"['PfizerBiontech', 'nhs', 'PfizerVaccine', 'Co..."
67929,Germany is reportedly seeking to buy 30 millio...,"['Germany', 'SputnikV', 'Russia']"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54904 entries, 0 to 69717
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      54904 non-null  object
 1   hashtags  54904 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [6]:
df.reset_index(drop=True, inplace=True)
df.head()

,text,hashtags
0,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech']
1,coronavirus SputnikV AstraZeneca PfizerBioNTec...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf..."
2,Explain to me again why we need a vaccine Bori...,"['whereareallthesickpeople', 'PfizerBioNTech']"
3,it is a bit sad to claim the fame for success ...,['vaccination']
4,There have not been many bright days in 2020 b...,"['BidenHarris', 'Election2020']"


In [7]:
# Extract vaccines from hashtags, and standardize the vaccine names
import ast
from rapidfuzz import process, fuzz
vaccines = ['PfizerBiontech', 'Sinopharm', 'Sinovac', 'Moderna', 'OxfordAstrazeneca', 'Covaxin', 'SputnikV']
for i in range(len(df)):
    hashtags = ast.literal_eval(df.hashtags[i])
    new_hashtags = []
    for element in hashtags:
        temp = process.extractOne(element, vaccines, scorer=fuzz.WRatio)
        if temp[1] > 80:
            new_hashtags.append(temp[0])
    if len(new_hashtags) < 1:
        df.hashtags[i] = None
    elif len(new_hashtags) == 1:
        df.hashtags[i] = new_hashtags[0]
    else:
        df.hashtags[i] = new_hashtags
df.head()

,text,hashtags
0,Same folks said daikon paste could treat a cyt...,PfizerBiontech
1,coronavirus SputnikV AstraZeneca PfizerBioNTec...,"[SputnikV, OxfordAstrazeneca, PfizerBiontech, ..."
2,Explain to me again why we need a vaccine Bori...,PfizerBiontech
3,it is a bit sad to claim the fame for success ...,None
4,There have not been many bright days in 2020 b...,None


In [8]:
# Drop rows with no vaccine in hashtags
df.dropna(inplace=True)
df.head()

,text,hashtags
0,Same folks said daikon paste could treat a cyt...,PfizerBiontech
1,coronavirus SputnikV AstraZeneca PfizerBioNTec...,"[SputnikV, OxfordAstrazeneca, PfizerBiontech, ..."
2,Explain to me again why we need a vaccine Bori...,PfizerBiontech
5,Covid vaccine You getting it CovidVaccine covi...,"[PfizerBiontech, Moderna]"
7,while deaths are closing in on the 300000 mark...,PfizerBiontech


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44255 entries, 0 to 54903
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      44255 non-null  object
 1   hashtags  44255 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [10]:
# Duplicate the instances with multiple hashtags, so each instance will only have one vaccine type
df.reset_index(drop=True, inplace=True)
new_df = {'text': list(), 'vaccine': list()}
for i in range(len(df)):
    if type(df.hashtags[i]) != list:
        new_df['text'].append(df.text[i])
        new_df['vaccine'].append(df.hashtags[i])
    else:
        for element in df.hashtags[i]:
            new_df['text'].append(df.text[i])
            new_df['vaccine'].append(element)
df_new = pd.DataFrame(new_df)
df_new.head()

,text,vaccine
0,Same folks said daikon paste could treat a cyt...,PfizerBiontech
1,coronavirus SputnikV AstraZeneca PfizerBioNTec...,SputnikV
2,coronavirus SputnikV AstraZeneca PfizerBioNTec...,OxfordAstrazeneca
3,coronavirus SputnikV AstraZeneca PfizerBioNTec...,PfizerBiontech
4,coronavirus SputnikV AstraZeneca PfizerBioNTec...,Moderna


In [11]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50858 entries, 0 to 50857
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     50858 non-null  object
 1   vaccine  50858 non-null  object
dtypes: object(2)
memory usage: 794.8+ KB


In [12]:
df_new.vaccine.value_counts()

Moderna              16474
Covaxin               9073
PfizerBiontech        8152
SputnikV              7518
OxfordAstrazeneca     4544
Sinovac               2731
Sinopharm             2366
Name: vaccine, dtype: int64

In [13]:
df_new.sample(10)

,text,vaccine
13751,What do Pfizer BioNTech Moderna Johnson amp Jo...,PfizerBiontech
1304,SkyNews SkyNewsAdele A clear message that supp...,PfizerBiontech
40530,covaxin aiims COVID19Vaccine PMModi Coronaviru...,Covaxin
49889,India will start receiving Russias SputnikV Co...,SputnikV
12719,Now all BJP including GodiMedia should take Co...,Covaxin
27836,First batch of Sputnik V vaccine arrives in Ke...,SputnikV
27727,Juut as covaxin and bharatbiotech aae ramping ...,Covaxin
2881,Today I got my first shot of the vaccine Pfize...,PfizerBiontech
11191,Lufthansa to launch a First Class Corona Loung...,SputnikV
46605,My turn moderna KissingSeasonIsBack Walgreens...,Moderna
